In [57]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from haversine import haversine
from sklearn.model_selection import train_test_split
np.random.seed(42)
import scipy.stats as stats
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from datetime import timedelta  
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import math

from matplotlib import rcParams
rcParams.update({'figure.autolayout':True})

plt.style.use(["presentation"])

%matplotlib inline

In [58]:
df1=pd.read_pickle('./res_parking_permit_violations')

In [59]:
df2=pd.read_csv('./Residential_Parking_Permit_Blocks.csv')

In [60]:
features=['OBJECTID', 'SERVICECODEDESCRIPTION', 'ADDDATE', 'RESOLUTIONDATE', 'STREETADDRESS',
          'LATITUDE', 'LONGITUDE', 'ZIPCODE', 'WARD', 'DETAILS']

In [61]:
# Creating a smaller version of the dataframe with just the features that might be useful

df_short=df1[features]
df_short = df_short.copy()

# A. Create some initial date-related features

In [62]:
# Converting dates to Pandas Date-Time Format

df_short['ADDDATE']=df_short['ADDDATE'].map(pd.to_datetime)
df_short['RESOLUTIONDATE']=df_short['RESOLUTIONDATE'].map(pd.to_datetime)


In [63]:
trial=df_short['ADDDATE'][58]

In [64]:
trial.date

<function Timestamp.date>

In [84]:
# Date ticket issued

df_short['date']=df_short['ADDDATE'].map(lambda x: x.strftime('%Y-%d-%m'))

In [66]:
# Day of week ticket issued

df_short['day_of_week']=df_short['ADDDATE'].map(lambda x : x.weekday_name)

In [67]:
# Year ticket issued

df_short['year']=df_short['ADDDATE'].map(lambda x : x.year)

In [68]:
# Month parking violation occurred

df_short['month']=df_short['ADDDATE'].map(lambda x : x.month)

In [106]:
# Time in minutes to resolve complaint

df_short['resolvetime']=(df_short['RESOLUTIONDATE']-df_short['ADDDATE']).astype('timedelta64[h]')

In [70]:
trial=df_short.loc[58,'ADDDATE']

# B. Bin observations by block

In [72]:
# Code from Ben



import re

def get_address_block(address):
    try:
        m = re.findall('(\d+\s)', address)
        return(np.mean([float(x.strip()) for x in m]))
    except:
        return(np.nan)

df_short['BLOCKNUM']=df_short['STREETADDRESS'].apply(get_address_block)

# Code from Ben ends here

def round_address_block(address):
    try:
        address=address/100
        address=int(address)
        address=address*100
        return address
    except:
        return(np.nan)

df_short['BLOCKNUM']=df_short['BLOCKNUM'].map(round_address_block)

/anaconda3/envs/dsi/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [73]:
def get_street_name(address):
    try:
        m1=address.replace("BLOCK OF","")
        m2=re.findall('([0-9]+[TH|ST|RD]+|[A-Z]+)', m1)
        return " ".join(m2)
    except:
        return (np.nan)

df_short['STREET']=df_short['STREETADDRESS'].apply(get_street_name)

In [74]:
df_short=df_short.dropna(subset=['BLOCKNUM', 'STREET'])

In [75]:
df_short=df_short.copy()

In [76]:
df_short['GEOBIN']=df_short['BLOCKNUM'].apply(lambda x: str(int(x)))

In [77]:
df_short['GEOBIN']=df_short['GEOBIN'].apply(lambda x: x+ " BLOCK ")

In [78]:
df_short['BLOCK']=df_short['GEOBIN']+df_short['STREET']

# C. A few observations about the blocks

In [79]:
# Here's the list of the blocks and the number of citations issued

df_short['BLOCK'].value_counts(normalize=True)[0:50]

1400 BLOCK CRITTENDEN STREET NW       0.016447
200 BLOCK 3RD STREET NE               0.015235
1600 BLOCK KRAMER STREET NE           0.014716
500 BLOCK 24TH STREET NE              0.014196
0 BLOCK BATES STREET NW               0.010249
1300 BLOCK I STREET NE                0.009314
1500 BLOCK OLIVE STREET NE            0.008725
4500 BLOCK CLARK PLACE NW             0.008310
2200 BLOCK KEARNY STREET NE           0.007860
6200 BLOCK ND STREET NW               0.007098
900 BLOCK 6TH STREET NE               0.006856
1200 BLOCK U STREET SE                0.006683
1400 BLOCK KENNEDY STREET NW          0.006094
500 BLOCK E STREET NE                 0.005678
2000 BLOCK 4TH STREET NE              0.005367
0 BLOCK N STREET SW                   0.005263
500 BLOCK 5TH STREET NE               0.005228
1200 BLOCK D STREET SE                0.005194
5300 BLOCK DANA PLACE NW              0.005021
100 BLOCK THOMAS STREET NW            0.004432
1100 BLOCK U STREET SE                0.004363
600 BLOCK 14T

In [80]:
# Here, for example, is a count of citations on Sunday, by contrast

df_short.loc[df_short['day_of_week']=='Sunday']['BLOCK'].value_counts()[0:50]

1400 BLOCK SWANN STREET NW          19
100 BLOCK SEATON PLACE NW            7
900 BLOCK T STREET NW                6
1600 BLOCK 11TH STREET NW            6
1200 BLOCK E STREET SE               6
1100 BLOCK W STREET NW               5
1700 BLOCK 9TH STREET NW             4
1500 BLOCK MARION STREET NW          3
1300 BLOCK R STREET NW               3
1300 BLOCK DELAFIELD PLACE NW        2
1000 BLOCK GIRARD STREET NW          2
200 BLOCK 3RD STREET NE              2
700 BLOCK 7TH STREET SW              2
1400 BLOCK HALF STREET SW            2
5100 BLOCK SOUTHERN AVENUE SE        2
900 BLOCK 7TH STREET SW              2
3200 BLOCK 13TH STREET NW            2
1200 BLOCK SIMMS PLACE NE            2
900 BLOCK P STREET NW                2
1300 BLOCK Q STREET NW               2
1500 BLOCK QUEEN STREET NE           2
1400 BLOCK S STREET SE               2
900 BLOCK O STREET NW                2
1600 BLOCK KRAMER STREET NE          2
1500 BLOCK 1ST STREET SW             2
1500 BLOCK S STREET SE   

In [23]:
# Here, for example, is a count of citations in 2017

df_short.loc[df_short['year']==2017]['BLOCK'].value_counts()[0:50]

1200 BLOCK D STREET SE                126
1200 BLOCK U STREET SE                116
200 BLOCK 3RD STREET NE               111
600 BLOCK 24TH STREET NE              110
1500 BLOCK OLIVE STREET NE            105
900 BLOCK 6TH STREET NE               105
1400 BLOCK CRITTENDEN STREET NW        87
1700 BLOCK NEW HAMPSHIRE AVENUE NW     79
2200 BLOCK KEARNY STREET NE            77
500 BLOCK 24TH STREET NE               75
2400 BLOCK E STREET NE                 66
4500 BLOCK CLARK PLACE NW              63
900 BLOCK VARNUM STREET NE             62
1600 BLOCK OLIVE STREET NE             60
0 BLOCK BATES STREET NW                60
1300 BLOCK I STREET NE                 57
1600 BLOCK KRAMER STREET NE            56
300 BLOCK 5TH STREET NE                50
1200 BLOCK SAVANNAH PLACE SE           44
1700 BLOCK 14TH STREET SE              43
0 BLOCK HANOVER PLACE NW               42
0 BLOCK N STREET SW                    40
1400 BLOCK HALF STREET SW              39
4600 BLOCK QUARLES STREET NE      

In [24]:
# Here, for example, is a count of citations on 2016

df_short.loc[df_short['year']==2016]['BLOCK'].value_counts()[0:50]

200 BLOCK 3RD STREET NE              159
4500 BLOCK CLARK PLACE NW            113
0 BLOCK BATES STREET NW              104
1300 BLOCK I STREET NE                98
1400 BLOCK CRITTENDEN STREET NW       88
500 BLOCK 5TH STREET NE               79
1200 BLOCK U STREET SE                76
1400 BLOCK KENNEDY STREET NW          69
0 BLOCK N STREET SW                   68
600 BLOCK STREET NW                   66
2000 BLOCK 4TH STREET NE              57
500 BLOCK 24TH STREET NE              56
500 BLOCK E STREET NE                 51
5300 BLOCK ILLINOIS AVENUE NW         50
900 BLOCK VARNUM STREET NE            45
1500 BLOCK OLIVE STREET NE            42
1100 BLOCK I STREET NE                42
600 BLOCK INDEPENDENCE AVENUE SE      40
2200 BLOCK KEARNY STREET NE           40
100 BLOCK THOMAS STREET NW            38
600 BLOCK G STREET NE                 37
600 BLOCK 24TH STREET NE              37
1600 BLOCK KRAMER STREET NE           35
900 BLOCK 6TH STREET NE               35
1500 BLOCK SWANN

In [28]:
resolution=df_short.groupby(by='BLOCK').mean()

In [30]:
resolution.sort_values(by='resolvetime')

,OBJECTID,LATITUDE,LONGITUDE,ZIPCODE,year,month,resolvetime,BLOCKNUM
BLOCK,,,,,,,,
1700 BLOCK NEWTON STREET NW,7.972600e+05,38.933986,-77.040751,20010.0,2015.000000,2.000000,0.000000,1700.0
5000 BLOCK 13TH STREET NW,9.302990e+05,38.950968,-77.029423,20011.0,2015.000000,5.000000,0.000000,5000.0
600 BLOCK 12TH STREET NE,1.560387e+06,38.898129,-76.990249,20002.0,2017.000000,3.000000,1.000000,600.0
3000 BLOCK EARL PLACE NE,1.635732e+06,38.928560,-76.963102,20018.0,2017.000000,6.000000,1.000000,3000.0
3800 BLOCK TUNLAW ROAD NW,1.795612e+06,38.925075,-77.077045,20007.0,2017.000000,11.000000,1.000000,3800.0
1500 BLOCK LONGFELLOW STREET NW,1.693085e+06,38.957282,-77.034926,20011.0,2017.000000,8.000000,1.000000,1500.0
2700 BLOCK N STREET NW,1.489113e+06,38.906711,-77.056382,20007.0,2016.000000,12.000000,1.000000,2700.0
3200 BLOCK 8TH STREET SE,6.240190e+05,38.843006,-76.994998,20032.0,2014.000000,6.000000,1.000000,3200.0
2400 BLOCK MINNESOTA AVENUE SE,1.565499e+06,38.873230,-76.971948,20020.0,2017.000000,3.000000,1.000000,2400.0


In [87]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

dr = pd.date_range(start='2011-01-01', end='2018-12-31')

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())


In [96]:
# of tickets issued by year.  2018 is incomplete

df_short['year'].value_counts()

2017    8166
2016    6482
2015    5790
2014    4332
2018    4111
Name: year, dtype: int64

In [97]:
# of incidents by month

df_short['month'].value_counts()

5     3151
4     2922
6     2856
3     2809
9     2466
8     2455
2     2239
10    2229
1     2127
7     2102
11    1781
12    1744
Name: month, dtype: int64

In [98]:
df.columns

Index(['Date', 'Holiday'], dtype='object')

In [105]:
df_short.loc['DETAILS']

KeyError: 'the label [DETAILS] is not in the [index]'